In [3]:
# %matplotlib nbagg
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import time
import pickle
from datetime import datetime

sys.path.append("..")
# 自分のpath
os.chdir("/home/nakamura/HAI/main_server/hai")

import coloredlogs, logging
logger = logging.getLogger(__name__)
coloredlogs.install(level='CRITICAL', logger=logger)

import pymongo
from pymongo import MongoClient
from PIL import Image
from matplotlib import pylab as plt
from matplotlib.animation import ArtistAnimation
import matplotlib.animation as animation
import numpy as np
import scipy
from bson.objectid import ObjectId
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from notebooks.utils.utils import visualize, display_latest_image, display_image, print_time, strtime_to_epoch, epoch_to_strtime, UpdateDist, display_two_images
from controllers.learner.img2vec import NNFeatures
from controllers.vectorizer.person2vec import Person2Vec
import controllers.learner.datasets as ds
from controllers.dbreader.hue_koki_dbreader import HueDBReader, pair_images, extract_color, ExprHueDBReader
from controllers.tests.test0106 import Test0106

mongo = MongoClient('localhost', 20202).hai
from flask import Flask
app = Flask(__name__)
app.config.from_pyfile(filename="application.cfg")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [4]:
start = strtime_to_epoch("2018-01-06 04:05:00")
# end = strtime_to_epoch("2018-12-17 15:05:00")
end = 10e10

In [5]:
hue = Test0106('koki')

In [6]:
epoch_to_strtime(1515227783.59965)

'2018-01-06 17:36:23'

In [4]:
img0_cols = mongo.images.find({'cam_id': 'webcam0', 'time': {'$gt': start, '$lt': end}})
img1_cols = mongo.images.find({'cam_id': 'webcam1', 'time': {'$gt': start, '$lt': end}})
imgs_cols = np.array(pair_images(img0_cols, img1_cols))

In [45]:
%matplotlib nbagg
fig, (axL, axR) = plt.subplots(ncols=2, figsize=(10,4))
ud = UpdateDist(axL, axR, imgs_cols, human=True, pose=True)
ani = animation.FuncAnimation(fig, ud, frames=range(ud.length), interval=200)
ani.save('0106-ht.mp4', writer="ffmpeg")
# fig.show()

<IPython.core.display.Javascript object>

KeyError: 'detections'

In [10]:
prints = mongo.print.find({'time': {'$gt': start, '$lt': end}})

In [11]:
for p in prints:
    print(p)

{'text': 'wow', '_id': ObjectId('5a4fccfa1d41c879d8e1e504'), 'time': 1515179258.990909, 'user_name': 'koki'}
{'text': 'スタート', '_id': ObjectId('5a4fce051d41c879d8e1e581'), 'time': 1515179525.060217, 'user_name': 'koki'}
{'text': 'TV', '_id': ObjectId('5a4fce131d41c879d8e1e58c'), 'time': 1515179539.906976, 'user_name': 'koki'}
{'text': 'ソファで寝る', '_id': ObjectId('5a4fce371d41c879d8e1e59f'), 'time': 1515179575.8342004, 'user_name': 'koki'}
{'text': '外出', '_id': ObjectId('5a4fceae1d41c879d8e1e5eb'), 'time': 1515179694.9625509, 'user_name': 'koki'}
{'text': 'アイスを食べる', '_id': ObjectId('5a4fcf041d41c879d8e1e61f'), 'time': 1515179780.8865013, 'user_name': 'koki'}
{'text': 'スマホ', '_id': ObjectId('5a4fcfa01d41c879d8e1e678'), 'time': 1515179936.7902431, 'user_name': 'koki'}
{'text': 'pc', '_id': ObjectId('5a4fd0d81d41c879d8e1e729'), 'time': 1515180248.8999238, 'user_name': 'koki'}
{'text': 'アイス（机で）', '_id': ObjectId('5a4fd15b1d41c879d8e1e770'), 'time': 1515180379.7111242, 'user_name': 'koki'}
{'te

In [12]:
def get_data():
    X = []
    y = []
    for al in activity_log:
        start = strtime_to_epoch(al['start'])
        end = strtime_to_epoch(al['end'])
        img0_cols = mongo.images.find({'cam_id': 'webcam0', 'time': {'$gt': start, '$lt': end}})
        img1_cols = mongo.images.find({'cam_id': 'webcam1', 'time': {'$gt': start, '$lt': end}})
        imgs_cols = pair_images(img0_cols, img1_cols)
        y.extend([al['activity']] * len(imgs_cols))
        X.extend(imgs_cols)

    return X, y

In [13]:
activity_log = [
    {'activity': 'sleep', 'start': '2018-01-06 04:13:04', 'end': '2018-01-06 04:14:52'},
    {'activity': 'absence', 'start': '2018-01-06 04:15:01', 'end': '2018-01-06 04:16:22'},
    {'activity': 'enjoy', 'start': '2018-01-06 04:16:26', 'end': '2018-01-06 04:23:57'},
    {'activity': 'work', 'start': '2018-01-06 04:24:11', 'end': '2018-01-06 04:25:58'},
    # {'activity': 'enjoy', 'start': '2018-01-06 04:24:11', 'end': '2018-01-06 04:25:58'},
    {'activity': 'sleep', 'start': '2018-01-06 04:28:34', 'end': '2018-01-06 04:33:53'},
]

In [15]:
X, y = get_data()

In [29]:
X[0][0]['pose']

{'body': [], 'face': [], 'hand': []}

In [32]:
vectorizer = Person2Vec()

In [34]:
a, b = vectorizer.vectorize(X)

In [36]:
a.shape

(232, 108)

In [37]:
a[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [38]:
a[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [43]:
a[50]

array([  2.12784302e+02,   1.19824715e+02,   7.70094156e-01,
         1.67733093e+02,   1.22801109e+02,   6.82386279e-01,
         1.12009216e+02,   1.19839363e+02,   5.63141882e-01,
         7.48733597e+01,   2.16671326e+02,   6.32584929e-01,
         1.45308670e+02,   1.76603134e+02,   7.37864912e-01,
         2.20582672e+02,   1.24725731e+02,   5.55003881e-01,
         2.20593491e+02,   2.23511169e+02,   5.68396270e-01,
         2.30406006e+02,   1.99107010e+02,   7.28856921e-01,
         1.38437576e+02,   2.53843414e+02,   2.58499503e-01,
         1.55048630e+02,   2.65585266e+02,   9.82041731e-02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         2.05932526e+02,   2.45055161e+02,   4.06736493e-01,
         2.92985687e+02,   2.39189682e+02,   3.85753125e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         2.01044479e+02,   1.06128029e+02,   8.60820055e-01,
         2.22530563e+02,   1.06157150e+02,   7.69898355e-01,
         1.68726700e+02,

In [42]:
b[50]

array([  4.87782955,  11.1205225 ,   2.7641542 , ...,   0.        ,
         0.        ,   0.        ])

In [46]:
b[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [47]:
b[1]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [48]:
b[4]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [ ]:
pa_arr_train = np.concatenate([pose_arr_train, act_arr_train], axis=1)

In [49]:
import pickle

In [51]:
hoge = pickle.loads('semisup.pkl')

TypeError: a bytes-like object is required, not 'str'

In [52]:
with open('semisup.pkl', mode='rb') as f:
    hoge = pickle.load(f)

In [53]:
hoge

{'classes': array(['enjoy', 'etc', 'sleep', 'work'],
       dtype='<U5'),
 'classifier': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False)}

In [58]:
c = hoge['classifier']

In [57]:
pa = np.concatenate([a, b], axis=1)

In [59]:
c.predict(pa)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [60]:
c.classes_

array([0, 2, 3])